In [ ]:
from minibt import *


class owen(Strategy):
    params = dict(l1=.6, l2=.95, l3=.65, symbol="v2601_60")

    def __init__(self):
        self.min_start_length = 300
        self.data = self.get_kline(
            LocalDatas.get(self.params.symbol), height=300)
        self.gc = self.data.tradingview.G_Channels()
        self.ebsw = self.data.close.ebsw()
        self.macd = self.data.close.macd()
        self.adosc = self.data.close.ao()
        self.btindicatordataset.height=50
        self.buy_signal = self.ebsw.cross_up(self.params.l1)
        self.exit_buy_signal = self.ebsw.cross_up(
            self.params.l2) | self.ebsw.cross_down(self.params.l3)
        self.sell_signal=self.ebsw.cross_down(-self.params.l1)
        self.exit_sell_signal = self.ebsw.cross_down(
            -self.params.l2) | self.ebsw.cross_up(-self.params.l3)
        self.buy_signal.isplot=self.exit_buy_signal.isplot=self.sell_signal.isplot=self.exit_sell_signal.isplot=False

    def next(self):
        if not self.data.position:
            if self.buy_signal.new:
                self.data.buy()
            elif self.sell_signal.new:
                self.data.sell()
        elif self.data.position > 0 and self.exit_buy_signal.new:
            self.data.sell()
        elif self.data.position<0 and self.exit_sell_signal.new:
            self.data.buy()



# 单策略回测
if __name__ == "__main__":
    Bt(auto=False).addstrategy(owen).run(isplot=True,isreport=True,  report_cwd="./test", report_name="test")


In [ ]:
# 多策略回测
owen1 = owen.copy(params=dict(l1=.5, l2=.8, l3=.7, symbol="v2601_300"))
if __name__ == "__main__":
    Bt(auto=False).addstrategy(owen,owen1).run(isplot=True,isreport=True,  report_cwd="./test", report_name="test")